## Section 1: Setup & Environment Configuration

In [ ]:
import os
from pathlib import Path
from dotenv import dotenv_values

# 找到 .env 檔案
env_file = Path.cwd().parent.parent / ".env"
if not env_file.exists():
    env_file = Path.home() / "Documents/workspace/CasualTrader/backend/.env"

# 讀取並設定環境變數
if env_file.exists():
    env_vars = dotenv_values(env_file)
    for key, value in env_vars.items():
        os.environ[key] = value.strip('"').strip("'") if value else ""

# 取得 API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GITHUB_PERSONAL_ACCESS_TOKEN = os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

print(f"✓ .env 已載入: {env_file}")
print(f"✓ OPENAI_API_KEY: {OPENAI_API_KEY[:20]}...{OPENAI_API_KEY[-20:]}" if OPENAI_API_KEY else "✗ 未設定")
print(f"✓ GITHUB_PERSONAL_ACCESS_TOKEN: {GITHUB_PERSONAL_ACCESS_TOKEN[:20]}...{GITHUB_PERSONAL_ACCESS_TOKEN[-20:]}" if GITHUB_PERSONAL_ACCESS_TOKEN else "✗ 未設定")
print(f"✓ TAVILY_API_KEY: {TAVILY_API_KEY[:20]}...{TAVILY_API_KEY[-20:]}" if TAVILY_API_KEY else "✗ 未設定")

## Section 2: LiteLLM GitHub Copilot - Authentication & Basic Usage

In [ ]:
from litellm import completion, acompletion
import asyncio

# Test 1: Non-streaming completion with GitHub Copilot
# Note: On first run, LiteLLM will prompt with OAuth device code

def test_github_copilot_basic():
    """
    Test basic GitHub Copilot completion via LiteLLM.
    
    First execution triggers OAuth device flow:
    - LiteLLM displays a device code and verification URL
    - Visit the URL and enter the code
    - Authenticate with your GitHub account
    - OAuth tokens are cached for future use
    """

    model = "github_copilot/gemini-2.5-pro"

    print("\n=== Test 1: Basic Completion (Non-Streaming) ===")
    print(f"Model: {model}")
    print("Status: Awaiting response...\n")
    
    try:
        response = completion(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": "Write a Python function to calculate the Fibonacci sequence up to n numbers, just python code without any explanation",
                }
            ],
            # temperature=0.7,
            max_tokens=500,
            extra_headers={
                "editor-version": "vscode/1.85.1",
                "Copilot-Integration-Id": "vscode-chat",
            },
        )
        
        # print("Response received:")
        print("-" * 50)
        print(response["choices"][0]["message"]["content"])
        print("-" * 50)
        # print(f"\nUsage: {response.get('usage', 'N/A')}")
        
    except Exception as e:
        print(f"Error: {type(e).__name__}")
        print(f"Message: {str(e)}")
        print("\nNote: This is expected if:")
        print("  1. OAuth authentication is needed (first run)")
        print("  2. GitHub Copilot subscription is not active")
        print("  3. Network connectivity issue")

# Run the test
test_github_copilot_basic()

## Section 3: LiteLLM GitHub Copilot - Streaming

In [ ]:
from litellm import completion, acompletion


def test_github_copilot_streaming():
    """
    Test streaming responses from GitHub Copilot via LiteLLM.
    
    Streaming is useful for real-time feedback in UI applications.
    """

    model = "github_copilot/grok-code-fast-1"

    print("\n=== Test 2: Streaming Completion ===")
    print(f"Model: {model}")
    print("Status: Streaming response...\n")
    
    try:
        stream = completion(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": "Explain async/await in Python with a simple example"
                }
            ],
            stream=True,
            # temperature=0.8,
            max_tokens=300,
            extra_headers={
                "editor-version": "vscode/1.85.1",
                "Copilot-Integration-Id": "vscode-chat"
            }
        )
        
        print("Streaming response:")
        print("-" * 50)
        
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        
        print("\n" + "-" * 50)
        print(f"\nTotal characters received: {len(full_response)}")
        
    except Exception as e:
        print(f"Error: {type(e).__name__}: {str(e)}")

# Run the test
test_github_copilot_streaming()

## Section 4: Example - Using GitHub Copilot with Agents SDK

In [ ]:
# Example configuration for using GitHub Copilot with OpenAI Agents

from agents import Agent, Runner, function_tool, ModelSettings, gen_trace_id, trace
from agents.extensions.models.litellm_model import LitellmModel
import json


model = "github_copilot/grok-code-fast-1"
print(f"\nUsing model: {model}")

model_settings = ModelSettings(
    include_usage=True,
    extra_headers={
        "editor-version": "vscode/1.85.1",
        "Copilot-Integration-Id": "vscode-chat",
    }
)


@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is rainy."

agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    # model=LitellmModel(model=model, api_key=GITHUB_PERSONAL_ACCESS_TOKEN),
    model=LitellmModel(model=model),
    tools=[get_weather],
    model_settings=model_settings,
)

trace_id = gen_trace_id()
with trace(workflow_name="lab_agent", trace_id=trace_id):
    result = await Runner.run(agent, "What's the weather in Tokyo?")
    print(json.dumps(result.final_output, indent=2))

print("\n=== GitHub Copilot + Agents Configuration ===")

## Section 5: Agent with MCP Tool Integration

In [ ]:
from contextlib import AsyncExitStack
from agents import Agent, Runner, function_tool, ModelSettings, gen_trace_id, trace
from agents.extensions.models.litellm_model import LitellmModel
from agents.mcp import MCPServerStdio
import json
from datetime import datetime

model = "github_copilot/grok-code-fast-1"
print(f"\nUsing model: {model}")

# Initialize MCP servers
async def get_tavily_mcp():
    """
    初始化 MCP 伺服器並註冊到 exit stack

    Raises:
        Exception: 初始化失敗
    """
    global _exit_stack  # 修正 UnboundLocalError
    try:
        if not _exit_stack:
            _exit_stack = AsyncExitStack() 

        # Tavily MCP Server
        # 參數在 MCPServerStdio 的 params 內配置
        tavily_mcp = await _exit_stack.enter_async_context(
            MCPServerStdio(
                name="tavily_mcp",
                params={
                    "command": "npx",
                    "args": ["-y", "tavily-mcp@latest"],
                    "env": {"TAVILY_API_KEY": f"{TAVILY_API_KEY}"},
                },
                client_session_timeout_seconds=30.0,
            )
        )
        print("tavily_mcp server initialized")
        return tavily_mcp

    except Exception as e:
        print(f"Failed to initialize MCP server: {e}")
        # 如果初始化失敗，清理已創建的資源
        if _exit_stack:
            await _exit_stack.aclose()
            _exit_stack = None

tavily_mcp = await get_tavily_mcp()

agent = Agent(
    name="Analyst",
    instructions="You using web search to answer user queries.",
    model=LitellmModel(model=model),
    mcp_servers=[tavily_mcp],
    model_settings=ModelSettings(
        include_usage=True,
        extra_headers={
            "editor-version": "vscode/1.85.1",
            "Copilot-Integration-Id": "vscode-chat",
        },
    ),
)

trace_id = gen_trace_id()
with trace(workflow_name="lab_section_5", trace_id=trace_id):
    message = f"下週沖繩的天氣如何？ (目前時間：{datetime.now().strftime('%Y-%m-%d %H:%M')})"
    print(f"message: {message}")
    print("-" * 50)
    result = await Runner.run(agent, message)
    print(json.dumps(result.final_output, indent=2, ensure_ascii=False))

print("\n=== Agent with MCP Tool Integration ===")


Using model: github_copilot/grok-code-fast-1
tavily_mcp server initialized
message: 下週沖繩的天氣如何？ (目前時間：2025-10-30 00:08)
--------------------------------------------------
"下週（2025年10月31日至11月6日）沖繩的天氣預計溫暖舒適，RealFeel溫度約25-32°C（77-90°F），風力中等（9-15 mph），紫外線指數中等。10月底可能有高浪警報，11月初風向變化，可能帶來些許涼爽。整體天氣穩定，但偶爾可能有小雨或雲層。建議查閱AccuWeather或其他可靠來源獲取最新詳細預報。"

=== Agent with MCP Tool Integration ===
